<a href="https://colab.research.google.com/gist/mlaves/a60cbc5541bd6c9974358fbaad9e4c51/incremental_ik-panda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Incremental Inverse Kinematics for Franka Emika Panda 7DoF Robot

The 3D pose $ \mathbf{x} \in \mathbb{R}^{6} $ of the end effector (EE) is given by

$$
\mathbf{x} = \mathbf{f}(\mathbf{q}) ~ ,
$$

with $ n $ joint angles $ \mathbf{q} $ and direct kinematics $\mathbf{f} : \mathbb{R}^{n} \to \mathbb{R}^{6} $.
We are interested in solving the inverse kinematics $\mathbf{q} = \mathbf{f}^{-1}(\mathbf{x})$, which we can do analytically (if possible), numerically (e.g., solve non-linear least-squares with Levenberg-Marquardt algorithm) or, as described here, using *incremental inverse kinematics* (IIK).

In IIK, we linearize the direct kinematics at the current joint angle configuration $\mathbf{q}^{\ast}$

$$
\Delta \left. \mathbf{x}\right|_{\mathbf{x}^{\ast}} \propto \Delta \left. \mathbf{q}\right|_{\mathbf{q}^{\ast}} ~ ,
$$

which we can easily solve around $ (\mathbf{x}^{\ast}, \mathbf{q}^{\ast}) $ using the Jacobian $ \mathbf{J}_{\mathbf{f}} := ( \partial f_{i} / \partial q_{j} )_{i,j} $

$$
\Delta \left. \mathbf{x}\right|_{\mathbf{x}^{\ast}} = \mathbf{J}_{\mathbf{f}}(\mathbf{q}^{\ast}) \Delta \left. \mathbf{q}\right|_{\mathbf{q}^{\ast}} ~ .
$$

## Steps of Incremental Inverse Kinematics

Given: target pose $ \mathbf{x}^{(t)} $  
Sought: joint angles $ \mathbf{q}^{(t)} $

0. Define starting pose $ (\mathbf{x}^{(0)}, \mathbf{q}^{(0)}) $ and set up IIK: $ \Delta \left. \mathbf{x}\right|_{\mathbf{x}^{(0)}} = \mathbf{J}(\mathbf{q}^{(0)}) \Delta \left. \mathbf{q}\right|_{\mathbf{q}^{(0)}} $.
1. Determine the deviation $ \Delta \mathbf{x}^{(k)} $ relative to the target pose; e.g. $ (\mathbf{x}^{(t)} - \mathbf{x}^{(0)}) $.
2. Check for termination; e.g., $ \max ( \vert \Delta x^{(k)}_{i,j} \vert ) \leq \epsilon $.
3. Solve $ \Delta \mathbf{x}^{(k)} = \mathbf{J}(\mathbf{q}^{(k)}) \Delta \mathbf{q}^{(k)} $ for $ \Delta \mathbf{q}^{(k)} $, e.g., by $ \Delta \mathbf{x}^{(k)} = (\mathbf{J}(\mathbf{q}^{(k)}) \mathbf{J}(\mathbf{q}^{(k)})^{T})^{-1} \Delta \mathbf{q}^{(k)} $.
4. Calculate new joint angles $ \mathbf{q}^{(k+1)} = \mathbf{q}^{(k)} + \Delta \mathbf{q}^{(k)} $.
5. $ k \leftarrow k + 1 $, go to 2.

## Set Up Direct Kinematics and Solve Jacobian Symbolically

In case of the Panda robot, we have $ n=7 $ active joint angles $ \mathbf{q} = \left[ \theta_{1}, \ldots, \theta_{7} \right]^{\mathsf{T}} $.
We use `sympy` to solve the Jacobian symbollically.

In [1]:
from sympy import symbols, init_printing, Matrix, eye, sin, cos, pi
init_printing(use_unicode=True)

In [2]:
# create joint angles as symbols

q1, q2, q3, q4, q5, q6, q7 = symbols('theta_1 theta_2 theta_3 theta_4 theta_5 theta_6 theta_7')
joint_angles = [q1, q2, q3, q4, q5, q6, q7]

In [3]:
# construct symbolic direct kinematics from Craig's DH parameters
# see https://frankaemika.github.io/docs/control_parameters.html

dh_craig = [
    {'a':  0,      'd': 0.333, 'alpha':  0,  },
    {'a':  0,      'd': 0,     'alpha': -pi/2},
    {'a':  0,      'd': 0.316, 'alpha':  pi/2},
    {'a':  0.0825, 'd': 0,     'alpha':  pi/2},
    {'a': -0.0825, 'd': 0.384, 'alpha': -pi/2},
    {'a':  0,      'd': 0,     'alpha':  pi/2},
    {'a':  0.088,  'd': 0.107, 'alpha':  pi/2},
]

DK = eye(4)

for i, (p, q) in enumerate(zip(reversed(dh_craig), reversed(joint_angles))):
    d = p['d']
    a = p['a']
    alpha = p['alpha']

    ca = cos(alpha)
    sa = sin(alpha)
    cq = cos(q)
    sq = sin(q)

    transform = Matrix(
        [
            [cq, -sq, 0, a],
            [ca * sq, ca * cq, -sa, -d * sa],
            [sa * sq, cq * sa, ca, d * ca],
            [0, 0, 0, 1],
        ]
    )

    DK = transform @ DK

In [4]:
# test direct kinematics

DK.evalf(subs={
    'theta_1': 0,
    'theta_2': 0,
    'theta_3': 0,
    'theta_4': 0,
    'theta_5': 0,
    'theta_6': 0,
    'theta_7': 0,
})

⎡1.0   0     0    0.088⎤
⎢                      ⎥
⎢ 0   -1.0   0      0  ⎥
⎢                      ⎥
⎢ 0    0    -1.0  0.926⎥
⎢                      ⎥
⎣ 0    0     0     1.0 ⎦

## Reshape Direct Kinematics

Let $ \mathrm{DK} : \mathbb{R}^{n} \to \mathbb{R}^{4 \times 4} $ be the direct kinematics transformation matrix (EE pose matrix), with

$$
\mathrm{DK} =
\left[
\begin{matrix}
a_{11} & a_{12} & a_{13} & a_{14} \\
a_{21} & a_{22} & a_{23} & a_{24} \\
a_{31} & a_{32} & a_{33} & a_{34} \\
0 & 0 & 0 & 1
\end{matrix}
\right] ~ .
$$

Instead of computing Euler angles from $ \mathrm{DK} $, we simply crop the last row and flatten column-wise:
$$
\mathrm{A} = \left[ a_{11}, a_{21}, a_{31}, \ldots, a_{34} \right]^{\mathsf{T}}, A \in \mathbb{R}^{12 \times 1} ~ .
$$

In [5]:
A = DK[0:3, 0:4]  # crop last row
A = A.transpose().reshape(12,1)  # reshape to column vector A = [a11, a21, a31, ..., a34]

Q = Matrix(joint_angles)
J = A.jacobian(Q)  # compute Jacobian symbolically

## From Symbolical to Numerical

We can now move to numerical computation for speed reasons.
With `lambdify`, we can create `numpy` functions from `sympy` expressions.
`numba` JIT compiles those functions to native code (optional).

In [6]:
import numpy as np
from sympy import lambdify
from numba import jit

A_lamb = jit(lambdify((q1, q2, q3, q4, q5, q6, q7), A, 'numpy'))
J_lamb = jit(lambdify((q1, q2, q3, q4, q5, q6, q7), J, 'numpy'))

## IIK Implementation

We are now ready to implement the actual IIK.

In [7]:
@jit
def incremental_ik(q, A, A_final, step=0.1, atol=1e-4):
    while True:
        delta_A = (A_final - A)
        if np.max(np.abs(delta_A)) <= atol:
            break
        J_q = J_lamb(q[0,0], q[1,0], q[2,0], q[3,0], q[4,0], q[5,0], q[6,0])
        J_q = J_q / np.linalg.norm(J_q)  # normalize Jacobian
        
        # multiply by step to interpolate between current and target pose
        delta_q = np.linalg.pinv(J_q) @ (delta_A*step)
        
        q = q + delta_q
        A = A_lamb(q[0,0], q[1,0],q[2,0],q[3,0],q[4,0],q[5,0],q[6,0])
    return q, np.max(np.abs(delta_A))

In [8]:
# define joint limits for the Panda robot
limits = [
    (-2.8973, 2.8973),
    (-1.7628, 1.7628),
    (-2.8973, 2.8973),
    (-3.0718, -0.0698),
    (-2.8973, 2.8973),
    (-0.0175, 3.7525),
    (-2.8973, 2.8973)
]

# create initial pose
q_init = np.array([l+(u-l)/2 for l, u in limits], dtype=np.float64).reshape(7, 1)
A_init = A_lamb(*(q_init.flatten()))
print(A_init.reshape(3, 4, order='F'))

[[ 0.9563065   0.          0.29236599  0.58193844]
 [ 0.         -1.          0.          0.        ]
 [ 0.29236599 -0.         -0.9563065   0.654902  ]]


In [9]:
# generate random final pose within joint limits

np.random.seed(0)

q_rand = np.array([np.random.uniform(l, u) for l, u in limits], dtype=np.float64).reshape(7, 1)
A_final = A_lamb(*(q_rand).flatten())
print(A_final.reshape(3, 4, order='F'))

[[ 0.21582232  0.95638546  0.19684405  0.55544547]
 [ 0.91784019 -0.26748917  0.29328985  0.50407938]
 [ 0.3331518   0.11737288 -0.93553914  0.33267676]]


In [10]:
q_final, _ = incremental_ik(q_init, A_init, A_final, atol=1e-6)
q_final.flatten()

array([ 0.44462097,  0.69891914,  0.37442864, -1.44809794, -0.21841531,
        2.45359754, -0.47850648])

## Test Runtime and Convergence Robustness

Randomly sample 10k target poses within joint limits to test runtime and convergence robustness.

In [14]:
%%timeit -n10000
q_rand = np.array([np.random.uniform(l, u) for l, u in limits], dtype=np.float64).reshape(7, 1)
incremental_ik(q_rand, A_init, A_final)

10000 loops, best of 5: 1.12 ms per loop
